## Manual Hamiltonian Construction

The code below calculates the cost Hamiltonians for 

S0 : x0 = 1

S1 : x0 + x1 = 1

S2 : x0 + x1 + x2 = 1

S3 : x0 + x1 + x2 + x3 = 1





In [305]:
import numpy as np

# Define matrices A, B, and I
A = np.array([
    [0, 0],
    [0, 1]
])

B = np.array([
    [1, 0],
    [0, 0]
])

I = np.eye(2)  # Identity matrix

# Calculate C using Kronecker product
C = np.kron(B, A) + np.kron(A, B)

# Compute S0
S0 = (np.eye(4) - C)

# Compute S1
S1 = np.kron(B, S0) + np.kron(A, C)

# Compute S2
S2 = np.kron(B, S1) + np.kron(A, np.kron(B, C) + np.kron(np.kron(A, I), B))

# Compute S3
S3 = np.kron(B, S2) + np.kron( A, np.kron(  B, np.kron(B, C) + np.kron(np.kron(A, I), B)  ) + np.kron(  A, np.kron(np.kron(I, I), B)  ) )

np.set_printoptions(threshold=np.inf, precision=3, suppress=True, linewidth=200)

print("S3:\n", S3)


S3:
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 

## Automatic Hamiltonian Construction

In [471]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import I, X, Y, Z

def A(i):
    return 0.5 * (qml.I(i) - qml.Z(i))

def B(i):
    return 0.5 * (qml.I(i) + qml.Z(i))

def C(i, j):
    return (A(i) @ B(j)) + (B(i) @ A(j)) 

def I(n):
    return qml.Identity(n)

def Q(n):
    recQ = []
    
    # First in Q recursive series 
    Q0 = (B(n) @ C(n-1, n-2)) + (A(n) @ I(n-1) @ B(n-2))
    recQ.append(Q0)
    
    if n > 2:
        for i in range(2, n):
            currentQmat = A(i)
            for j in range(0, i+1):
                currentQmat = currentQmat @ I(j+1)
            #currentQmat = (currentQmat @ I(i-1) @ B(i-2))
            #print(np.real(qml.matrix(currentQmat)))
            currentQmat = (B(i-2) @ recQ[i-2])
            recQ.append(currentQmat)
        
    return recQ[n-2]

def S(n):
    recS = []
    
    # First in S recursive series
    S0 = qml.I(n-1) - C(n-1, n-2)
    recS.append(S0)
    S1 = (B(n) @ S0) + (A(n) @ C(n-1, n-2))
    recS.append(S1)

    if n == 0:
        return recS[0]
    elif n == 1:
        return recS[1]
    else:
        for i in range(2, n+1):
            currentSmat = (B(i) @ recS[i-1]) + (A(i) @ Q(i-1))
            recS.append(currentSmat)
        
        return recS[n]

def constructCostH(n):
    return S(n)

# Test the function
n = 3
Q(n)
#print(np.real(qml.matrix(Q(n))))


(0.5 * (I(0) + Z(0))) @ ((0.5 * (I(3) + Z(3))) @ ((0.5 * (I(2) + -1 * Z(2))) @ (0.5 * (I(1) + Z(1))) + (0.5 * (I(2) + Z(2))) @ (0.5 * (I(1) + -1 * Z(1)))) + (0.5 * (I(3) + -1 * Z(3))) @ I(2) @ (0.5 * (I(1) + Z(1))))

In [474]:
coeff = [1, 1]
obs = []

# Alternating Ones and Zeros 
obs.append((
  (0.5 * (I(2) + -1 * Z(2))) @ I(1) @ I(2) @ I(3) @ I(1) @ (0.5 * (I(0) + Z(0)))) 
)

# First portion
obs.append(
  (0.5 * (I(0) + Z(0))) @ ((0.5 * (I(3) + Z(3))) @ ((0.5 * (I(2) + -1 * Z(2))) @ (0.5 * (I(1) + Z(1))) + (0.5 * (I(2) + Z(2))) @ (0.5 * (I(1) + -1 * Z(1)))) + (0.5 * (I(3) + -1 * Z(3))) @ I(2) @ (0.5 * (I(1) + Z(1))))
)

hamiltonian = qml.Hamiltonian(coeff, obs)
print(np.real(qml.matrix(hamiltonian)))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
